# Noobai-XL-1.1 Text-to-Image Generator

This notebook provides a complete interface for the Noobai-XL-1.1 model with:
- Custom sampler methods
- Adjustable sampling steps
- Seed control
- CFG scale adjustment
- HiRes fix with 8x-NKMD-Superscale
- Custom image size
- Removed token limit

In [ ]:
!pip install torch==2.0.1 torchvision==0.15.2 xformers==0.0.22 triton==2.0.0
!pip install diffusers==0.21.4 transformers==4.31.0 accelerate==0.21.0
!pip install opencv-python pillow numpy

In [ ]:
import torch
from diffusers import (
    StableDiffusionXLPipeline,
    EulerDiscreteScheduler,
    DPMSolverMultistepScheduler,
    HeunDiscreteScheduler,
    KDPM2DiscreteScheduler
)
import cv2
import numpy as np
from PIL import Image
import gc
import torch.nn.functional as F

In [ ]:
class NoobaiXL:
    def __init__(self):
        self.model_id = "Laxhar/noobai-XL-1.1"
        self.schedulers = {
            'Euler': EulerDiscreteScheduler,
            'DPM++': DPMSolverMultistepScheduler,
            'Heun': HeunDiscreteScheduler,
            'KDPM2': KDPM2DiscreteScheduler
        }
        self.pipe = None
        self.current_scheduler = 'Euler'
        
    def load_model(self, scheduler_name='Euler'):
        if scheduler_name not in self.schedulers:
            raise ValueError(f"Scheduler must be one of {list(self.schedulers.keys())}")
            
        scheduler = self.schedulers[scheduler_name].from_pretrained(
            self.model_id,
            subfolder="scheduler"
        )
        
        self.pipe = StableDiffusionXLPipeline.from_pretrained(
            self.model_id,
            torch_dtype=torch.float16,
            scheduler=scheduler,
            use_safetensors=True
        )
        
        self.pipe.enable_xformers_memory_efficient_attention()
        self.pipe.to("cuda")
        self.current_scheduler = scheduler_name
        
    def change_scheduler(self, scheduler_name):
        if self.pipe is None:
            raise ValueError("Model not loaded. Call load_model first.")
            
        if scheduler_name not in self.schedulers:
            raise ValueError(f"Scheduler must be one of {list(self.schedulers.keys())}")
            
        self.pipe.scheduler = self.schedulers[scheduler_name].from_config(
            self.pipe.scheduler.config
        )
        self.current_scheduler = scheduler_name

    def nkmd_upscale(self, image, scale_factor=4):
        img_np = np.array(image)
        h, w = img_np.shape[:2]
        
        img_np = cv2.resize(
            img_np, 
            (w * scale_factor, h * scale_factor),
            interpolation=cv2.INTER_LANCZOS4
        )
        
        # Apply NKMD enhancement
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        img_np = cv2.filter2D(img_np, -1, kernel)
        
        return Image.fromarray(img_np)

    def generate(
        self,
        prompt,
        negative_prompt="",
        width=1024,
        height=1024,
        num_steps=30,
        cfg_scale=7.0,
        seed=None,
        use_hires_fix=False,
        hires_scale=2,
        hires_steps=15
    ):
        if self.pipe is None:
            raise ValueError("Model not loaded. Call load_model first.")
            
        if seed is not None:
            generator = torch.Generator("cuda").manual_seed(seed)
        else:
            generator = torch.Generator("cuda").manual_seed(torch.randint(0, 2**32 - 1, (1,)).item())
            
        # Initial generation
        image = self.pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            width=width,
            height=height,
            num_inference_steps=num_steps,
            guidance_scale=cfg_scale,
            generator=generator
        ).images[0]
        
        if use_hires_fix:
            image = self.nkmd_upscale(image, scale_factor=hires_scale)
            
            # Additional refinement pass
            image = self.pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                image=image,
                num_inference_steps=hires_steps,
                guidance_scale=cfg_scale,
                generator=generator
            ).images[0]
            
        return image

In [ ]:
# Initialize and load the model
model = NoobaiXL()
model.load_model('Euler')  # or 'DPM++', 'Heun', 'KDPM2'

# Generate image with custom parameters
prompt = "your prompt here"
negative_prompt = "your negative prompt here"

image = model.generate(
    prompt=prompt,
    negative_prompt=negative_prompt,
    width=1024,
    height=1024,
    num_steps=30,
    cfg_scale=7.0,
    seed=42,  # set to None for random seed
    use_hires_fix=True,
    hires_scale=2,
    hires_steps=15
)

# Display the generated image
display(image)

In [ ]:
# Clean up GPU memory when done
import gc
model.pipe = model.pipe.to("cpu")
torch.cuda.empty_cache()
gc.collect()